In [1]:
import nrrd # pip install pynrrd
import nibabel as nib # pip install nibabel
import numpy as np
import os
import itk

In [2]:
def find_leafes( root ):
    """ finds folders with no subfolders """
    for root, dirs, files in os.walk(root):
        if not dirs: # can't go deeper
            return root

In [3]:
number = 1051

In [4]:
root_dir = r'E:\Pancreas_BB\Bounding boxes\BB Tumor(+), Vaat(-)\{}\par'.format(number)
image_path = os.path.join(root_dir, '11 Unnamed Series.nrrd')

In [5]:
val = 1051
annDir = find_leafes(r'E:\Pancreas_BB\Detail annotaties\Detail Vaat(-)\{}'.format(val))

In [6]:
annDir

'E:\\Pancreas_BB\\Detail annotaties\\Detail Vaat(-)\\1051\\par\\Patient 1051_1051\\No study description\\Patient 1051 10 S__(10-30-2020_15-49-03-7515)'

In [7]:
# Setup ITK
# PixelType = itk.ctype('signed short')
# Dimension = 3
ImageType = itk.Image[itk.F, 3]


In [8]:
# Read data
reader = itk.ImageFileReader[itk.Image[itk.F, 3]].New(FileName=image_path)
reader.Update()


In [9]:
# Annotation directory
annotations = ['Gastroduodenalis', 'AMS', 'Aorta', 'Pancreas', 'Splenic vein', 'Truncus', 'Vena Cava', 'Vena porta', 'VMI', 'Tumour']
#annotations = ['Tumour']

In [10]:
all_labels = []
for ann in annotations:

    print('ann:', ann)

    labelfilelist = [file for file in os.listdir(annDir) if ann.lower() in file.lower()]

    if len(labelfilelist) > 0:

        labelfile = labelfilelist[0]

        print('labelfile: ', labelfile)
        labelpath = os.path.join(annDir, labelfile)

        MeshType = itk.Mesh[itk.F, 3]
        meshReader = itk.MeshFileReader[MeshType].New()
        meshReader.SetFileName(labelpath)
        meshReader.Update()

        #print('Meshreaderout: ', meshReader.GetOutput())

        ImageType = itk.Image[itk.F, 3]

        filter = itk.TriangleMeshToBinaryImageFilter[MeshType, ImageType].New()
        filter.SetInput(meshReader.GetOutput())
        filter.SetInfoImage(reader.GetOutput())
        filter.Update()

        #print('filterout: ', filter.GetOutput())
        image = np.array(itk.array_from_image(filter.GetOutput())).astype(np.bool)
        all_labels.append(image)

    else:
        
        imsize = reader.GetOutput().GetLargestPossibleRegion().GetSize()

        image = np.zeros((imsize[2], imsize[1], imsize[0])).astype(np.bool)

        all_labels.append(image)

ann: Gastroduodenalis
labelfile:  Patient 1051 10 S_a. Gastroduodenalis_(10-30-2020_15-49-03-7515).vtk
ann: AMS
labelfile:  Patient 1051 10 S_AMS_(10-30-2020_15-49-03-7515).vtk
ann: Aorta
labelfile:  Patient 1051 10 S_Aorta_(10-30-2020_15-49-03-7515).vtk
ann: Pancreas
labelfile:  Patient 1051 10 S_Pancreas_(10-30-2020_15-49-03-7515).vtk
ann: Splenic vein
labelfile:  Patient 1051 10 S_Splenic vein_(10-30-2020_15-49-03-7515).vtk
ann: Truncus
labelfile:  Patient 1051 10 S_Truncus, a. Hep + a. Splen_(10-30-2020_15-49-03-7515).vtk
ann: Vena Cava
labelfile:  Patient 1051 10 S_Vena cava_(10-30-2020_15-49-03-7515).vtk
ann: Vena porta
labelfile:  Patient 1051 10 S_Vena porta + VMS_(10-30-2020_15-49-03-7515).vtk
ann: VMI
labelfile:  Patient 1051 10 S_VMI_(10-30-2020_15-49-03-7515).vtk
ann: Tumour
labelfile:  Patient 1051 10 S_Tumour_(10-30-2020_15-49-03-7515).vtk


In [11]:
# process label
print([ls.shape for ls in all_labels])
labels = np.stack(all_labels)


[(433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512), (433, 512, 512)]


In [13]:
labelmap = labels.astype(np.uint8)

In [18]:
new_labelmap = np.concatenate((np.zeros((1,433, 512, 512)).astype(np.uint8), labelmap), axis=0)

In [20]:

labelmap = new_labelmap.argmax(axis=0)
labelmap = labelmap.astype(np.float32)
#labelmap[:,:,] = 0.
labelmap[-1, :,:] = 0.

In [22]:
np.unique(labelmap)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
      dtype=float32)

In [21]:

print(labelmap.shape)

labelType = itk.Image[itk.F, 3]
labelmap = np.ascontiguousarray(labelmap)
itk_image = itk.GetImageFromArray(labelmap)
itk_image.SetMetaDataDictionary(reader.GetMetaDataDictionary())
itk_image.Update()

metadata = reader.GetMetaDataDictionary()
print('metadata: ', metadata)


(433, 512, 512)
metadata:  Dictionary use_count: 4
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[0]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[1]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[2]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_space  [UNKNOWN_PRINT_CHARACTERISTICS]



In [23]:

writer = itk.ImageFileWriter[labelType].New()
writer.SetInput(itk_image)
writer.SetFileName(os.path.join(root_dir, 'dat_001.nii.gz'))
writer.Update()

In [17]:
# for i in range(0,10):
#     labelmap = labels[i].astype(np.float32)
#     labelmap[-1, :,:] = 0.
#     print(labelmap.shape)

#     labelType = itk.Image[itk.F, 3]
#     labelmap = np.ascontiguousarray(labelmap)
#     itk_image = itk.GetImageFromArray(labelmap)
#     itk_image.SetMetaDataDictionary(reader.GetMetaDataDictionary())
#     itk_image.Update()

#     metadata = reader.GetMetaDataDictionary()
#     print('metadata: ', metadata)

#     writer = itk.ImageFileWriter[labelType].New()
#     writer.SetInput(itk_image)
#     name = 'dat_00{}.nii.gz'.format(i+1)
#     writer.SetFileName(os.path.join(root_dir, name))
#     writer.Update()

(433, 512, 512)
metadata:  Dictionary use_count: 4
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[0]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[1]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[2]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_space  [UNKNOWN_PRINT_CHARACTERISTICS]

(433, 512, 512)
metadata:  Dictionary use_count: 6
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[0]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[1]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[2]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_space  [UNKNOWN_PRINT_CHARACTERISTICS]

(433, 512, 512)
metadata:  Dictionary use_count: 6
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHA

In [ ]:
# labelmap = labels.astype(np.uint8)
# labelmap = labelmap.argmax(axis=0)
# labelmap = labelmap.astype(np.float32)
# labelmap[-1, :,:] = 0.

In [63]:

print(labelmap.shape)

labelType = itk.Image[itk.F, 3]
labelmap = np.ascontiguousarray(labelmap)
itk_image = itk.GetImageFromArray(labelmap)
itk_image.SetMetaDataDictionary(reader.GetMetaDataDictionary())
itk_image.Update()

metadata = reader.GetMetaDataDictionary()
print('metadata: ', metadata)



(433, 512, 512)
metadata:  Dictionary use_count: 6
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[0]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[1]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[2]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_space  [UNKNOWN_PRINT_CHARACTERISTICS]



In [64]:

writer = itk.ImageFileWriter[labelType].New()
writer.SetInput(itk_image)
writer.SetFileName(os.path.join(root_dir, '010.nii.gz'))
writer.Update()

(433, 512, 512)

In [20]:
# Backgound label
labelType = itk.Image[itk.F, 3]
labelmap = np.ascontiguousarray(np.zeros((433, 512, 512)).astype(np.float32))
itk_image = itk.GetImageFromArray(np.zeros((433, 512, 512)).astype(np.float32))
itk_image.SetMetaDataDictionary(reader.GetMetaDataDictionary())
itk_image.Update()

metadata = reader.GetMetaDataDictionary()
print('metadata: ', metadata)
writer = itk.ImageFileWriter[labelType].New()
writer.SetInput(itk_image)
writer.SetFileName(os.path.join(root_dir, '000.nii.gz'))
writer.Update()

metadata:  Dictionary use_count: 6
ITK_InputFilterName  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_direction  [UNKNOWN_PRINT_CHARACTERISTICS]
ITK_original_spacing  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[0]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[1]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_kinds[2]  [UNKNOWN_PRINT_CHARACTERISTICS]
NRRD_space  [UNKNOWN_PRINT_CHARACTERISTICS]

